<a href="https://colab.research.google.com/github/crissyro/base-of-DS/blob/master/pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

# Pyspark practice


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
! cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/ # файл kaggle.json можно положить в любую папку в Drive, главное указать актуальный адрес в первом аргументе команды cp
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'riiid-test-answer-prediction'
! mkdir data
! unzip riiid-test-answer-prediction.zip -d data

Mounted at /content/drive
 99% 1.28G/1.29G [00:14<00:00, 153MB/s]
100% 1.29G/1.29G [00:14<00:00, 96.4MB/s]
Archive:  riiid-test-answer-prediction.zip
  inflating: data/example_sample_submission.csv  
  inflating: data/example_test.csv   
  inflating: data/lectures.csv       
  inflating: data/questions.csv      
  inflating: data/riiideducation/__init__.py  
  inflating: data/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: data/train.csv          


In [ ]:
! pip install pyspark
! pip install pyarrow

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=4f83cb855c8bad8e8d7967fc3ae5c03c41ecdd4866da011d449b1d2f63b60180
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [18]:
from pyspark.sql import SparkSession
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" # без этой строчки у нас будет возникать постоянное предупреждение с просьбой установить эту переменную в значение 1, что мы заранее и делаем


spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark

In [ ]:
df = spark.read.csv('data/train.csv', header=True, inferSchema=True)

df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- content_id: integer (nullable = true)
 |-- content_type_id: integer (nullable = true)
 |-- task_container_id: integer (nullable = true)
 |-- user_answer: integer (nullable = true)
 |-- answered_correctly: integer (nullable = true)
 |-- prior_question_elapsed_time: double (nullable = true)
 |-- prior_question_had_explanation: boolean (nullable = true)



In [ ]:
df.show()

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       null|                          null|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [ ]:
from pyspark.sql.types import IntegerType

df = df.withColumn('prior_question_had_explanation', df['prior_question_had_explanation'].cast(IntegerType()))
df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- content_id: integer (nullable = true)
 |-- content_type_id: integer (nullable = true)
 |-- task_container_id: integer (nullable = true)
 |-- user_answer: integer (nullable = true)
 |-- answered_correctly: integer (nullable = true)
 |-- prior_question_elapsed_time: double (nullable = true)
 |-- prior_question_had_explanation: integer (nullable = true)



In [ ]:
df.pandas_api().isna().mean()

row_id                            0.0
timestamp                         0.0
user_id                           0.0
content_id                        0.0
content_type_id                   0.0
task_container_id                 0.0
user_answer                       0.0
answered_correctly                0.0
prior_question_elapsed_time       0.0
prior_question_had_explanation    0.0
dtype: float64

In [ ]:
df = df.dropna()
df.pandas_api().isna().sum()

row_id                            0
timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
answered_correctly                0
prior_question_elapsed_time       0
prior_question_had_explanation    0
dtype: int64

In [19]:
table = [
    ['Dima', '8-800-555-35-35', 'Iphone'],
    ['Vanya', '8-777-777-77-77', 'Samsung'],
    ['Arsen', '8-555-555-55-55', 'Xiaomi'],
    ['Danil', '8-616-616-61-61', 'Iphone'],
    ['xzczzx', '213', 'Mezu']
]

df = spark.createDataFrame(table, ['name', 'mobile number', 'phone mark'])
df

DataFrame[name: string, mobile number: string, phone mark: string]

In [20]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mobile number: string (nullable = true)
 |-- phone mark: string (nullable = true)



In [21]:
df.show()

+------+---------------+----------+
|  name|  mobile number|phone mark|
+------+---------------+----------+
|  Dima|8-800-555-35-35|    Iphone|
| Vanya|8-777-777-77-77|   Samsung|
| Arsen|8-555-555-55-55|    Xiaomi|
| Danil|8-616-616-61-61|    Iphone|
|xzczzx|            213|      Mezu|
+------+---------------+----------+



In [22]:
df.columns

['name', 'mobile number', 'phone mark']

In [23]:
df.columns[1] = 'phone number'
df.show()

+------+---------------+----------+
|  name|  mobile number|phone mark|
+------+---------------+----------+
|  Dima|8-800-555-35-35|    Iphone|
| Vanya|8-777-777-77-77|   Samsung|
| Arsen|8-555-555-55-55|    Xiaomi|
| Danil|8-616-616-61-61|    Iphone|
|xzczzx|            213|      Mezu|
+------+---------------+----------+



In [24]:
df.toPandas()

,name,mobile number,phone mark
0,Dima,8-800-555-35-35,Iphone
1,Vanya,8-777-777-77-77,Samsung
2,Arsen,8-555-555-55-55,Xiaomi
3,Danil,8-616-616-61-61,Iphone
4,xzczzx,213,Mezu


In [25]:
df.toJSON()

MapPartitionsRDD[46] at toJavaRDD at NativeMethodAccessorImpl.java:0

In [26]:
df.count()

5

In [28]:
df.pandas_api().isna().sum()

name             0
mobile number    0
phone mark       0
dtype: int64

In [30]:
df.select('name', 'phone mark').show()

+------+----------+
|  name|phone mark|
+------+----------+
|  Dima|    Iphone|
| Vanya|   Samsung|
| Arsen|    Xiaomi|
| Danil|    Iphone|
|xzczzx|      Mezu|
+------+----------+



In [32]:
import pyspark.sql.functions as f

In [34]:
df.filter(f.col('name').startswith('D')).show()

+-----+---------------+----------+
| name|  mobile number|phone mark|
+-----+---------------+----------+
| Dima|8-800-555-35-35|    Iphone|
|Danil|8-616-616-61-61|    Iphone|
+-----+---------------+----------+



In [38]:
df.filter('name like "V_%"').show()

+-----+---------------+----------+
| name|  mobile number|phone mark|
+-----+---------------+----------+
|Vanya|8-777-777-77-77|   Samsung|
+-----+---------------+----------+



In [51]:
table_salary = [
    ['Dima', 'Senior Fullstack developer', 4500],
    ['Roma', 'Senior Data Scientist', 5500],
    ['Danil', 'Middle 1C developer', 2500],
    ['Vanya', 'Junior Cybersecuirity', 800],
    ['Andrey', 'Middle Backend developer', 2330]
]

df_salary = spark.createDataFrame(table_salary, ['name_person', 'job', 'salary'])
df_salary.printSchema()

root
 |-- name_person: string (nullable = true)
 |-- job: string (nullable = true)
 |-- salary: long (nullable = true)



In [52]:
df_salary.show(truncate=False)

+-----------+--------------------------+------+
|name_person|job                       |salary|
+-----------+--------------------------+------+
|Dima       |Senior Fullstack developer|4500  |
|Roma       |Senior Data Scientist     |5500  |
|Danil      |Middle 1C developer       |2500  |
|Vanya      |Junior Cybersecuirity     |800   |
|Andrey     |Middle Backend developer  |2330  |
+-----------+--------------------------+------+



In [57]:
full_df = df.join(df_salary, df_salary['name_person'] == df['name'], 'inner').drop('name_person')
full_df.show(truncate=False)

+-----+---------------+----------+--------------------------+------+
|name |mobile number  |phone mark|job                       |salary|
+-----+---------------+----------+--------------------------+------+
|Danil|8-616-616-61-61|Iphone    |Middle 1C developer       |2500  |
|Dima |8-800-555-35-35|Iphone    |Senior Fullstack developer|4500  |
|Vanya|8-777-777-77-77|Samsung   |Junior Cybersecuirity     |800   |
+-----+---------------+----------+--------------------------+------+



In [58]:
x = df.join(df_salary, df_salary['name_person'] == df['name'], 'left').drop('name_person')
x.show(truncate=False)

+------+---------------+----------+--------------------------+------+
|name  |mobile number  |phone mark|job                       |salary|
+------+---------------+----------+--------------------------+------+
|Vanya |8-777-777-77-77|Samsung   |Junior Cybersecuirity     |800   |
|Dima  |8-800-555-35-35|Iphone    |Senior Fullstack developer|4500  |
|Arsen |8-555-555-55-55|Xiaomi    |NULL                      |NULL  |
|Danil |8-616-616-61-61|Iphone    |Middle 1C developer       |2500  |
|xzczzx|213            |Mezu      |NULL                      |NULL  |
+------+---------------+----------+--------------------------+------+



In [59]:
x = df.join(df_salary, df_salary['name_person'] == df['name'], 'right').drop('name_person')
x.show(truncate=False)

+-----+---------------+----------+--------------------------+------+
|name |mobile number  |phone mark|job                       |salary|
+-----+---------------+----------+--------------------------+------+
|NULL |NULL           |NULL      |Senior Data Scientist     |5500  |
|Dima |8-800-555-35-35|Iphone    |Senior Fullstack developer|4500  |
|NULL |NULL           |NULL      |Middle Backend developer  |2330  |
|Vanya|8-777-777-77-77|Samsung   |Junior Cybersecuirity     |800   |
|Danil|8-616-616-61-61|Iphone    |Middle 1C developer       |2500  |
+-----+---------------+----------+--------------------------+------+



In [61]:
x = df.join(df_salary, df_salary['name_person'] == df['name'], 'full').drop('name_person')
x.show(truncate=False)

+------+---------------+----------+--------------------------+------+
|name  |mobile number  |phone mark|job                       |salary|
+------+---------------+----------+--------------------------+------+
|NULL  |NULL           |NULL      |Middle Backend developer  |2330  |
|Arsen |8-555-555-55-55|Xiaomi    |NULL                      |NULL  |
|Danil |8-616-616-61-61|Iphone    |Middle 1C developer       |2500  |
|Dima  |8-800-555-35-35|Iphone    |Senior Fullstack developer|4500  |
|NULL  |NULL           |NULL      |Senior Data Scientist     |5500  |
|Vanya |8-777-777-77-77|Samsung   |Junior Cybersecuirity     |800   |
|xzczzx|213            |Mezu      |NULL                      |NULL  |
+------+---------------+----------+--------------------------+------+



In [62]:
full_df.show(truncate=False)

+-----+---------------+----------+--------------------------+------+
|name |mobile number  |phone mark|job                       |salary|
+-----+---------------+----------+--------------------------+------+
|Danil|8-616-616-61-61|Iphone    |Middle 1C developer       |2500  |
|Dima |8-800-555-35-35|Iphone    |Senior Fullstack developer|4500  |
|Vanya|8-777-777-77-77|Samsung   |Junior Cybersecuirity     |800   |
+-----+---------------+----------+--------------------------+------+



In [65]:
new_df = spark.createDataFrame([
    ['Igor', None, 'Iphone', 'HR', 600],
    ['Petya', '7-930-444-54-54', 'Samsung', 'Junior Frontend developer', 630],
    ['Anton', None, None, 'Middle C++ developer', 2460],
    ['Vlad', None, 'Iphone', 'Senior Blockchain engineer', 9500]
], ['name', 'mobile number', 'phone mark', 'job', 'salary'])

new_df.show(truncate=False)

+-----+---------------+----------+--------------------------+------+
|name |mobile number  |phone mark|job                       |salary|
+-----+---------------+----------+--------------------------+------+
|Igor |NULL           |Iphone    |HR                        |600   |
|Petya|7-930-444-54-54|Samsung   |Junior Frontend developer |630   |
|Anton|NULL           |NULL      |Middle C++ developer      |2460  |
|Vlad |NULL           |Iphone    |Senior Blockchain engineer|9500  |
+-----+---------------+----------+--------------------------+------+



In [66]:
df = full_df.union(new_df)
df.show(truncate=False)

+-----+---------------+----------+--------------------------+------+
|name |mobile number  |phone mark|job                       |salary|
+-----+---------------+----------+--------------------------+------+
|Danil|8-616-616-61-61|Iphone    |Middle 1C developer       |2500  |
|Dima |8-800-555-35-35|Iphone    |Senior Fullstack developer|4500  |
|Vanya|8-777-777-77-77|Samsung   |Junior Cybersecuirity     |800   |
|Igor |NULL           |Iphone    |HR                        |600   |
|Petya|7-930-444-54-54|Samsung   |Junior Frontend developer |630   |
|Anton|NULL           |NULL      |Middle C++ developer      |2460  |
|Vlad |NULL           |Iphone    |Senior Blockchain engineer|9500  |
+-----+---------------+----------+--------------------------+------+



In [67]:
df.select(f.sum('salary')).show()

+-----------+
|sum(salary)|
+-----------+
|      20990|
+-----------+



In [70]:
df.select(f.sum('salary').alias('sum_salary'),
          f.min('salary').alias('min_salary'),
          f.max('salary').alias('max_salary'),
          f.mean('salary').alias('mean_salary'),
          f.variance('salary').alias('var_salary')
).show()

+----------+----------+----------+------------------+-------------------+
|sum_salary|min_salary|max_salary|       mean_salary|         var_salary|
+----------+----------+----------+------------------+-------------------+
|     20990|       600|      9500|2998.5714285714284|1.020974761904762E7|
+----------+----------+----------+------------------+-------------------+



In [78]:
df.filter('salary > 2000').select('name', 'job').show()

+-----+--------------------+
| name|                 job|
+-----+--------------------+
|Danil| Middle 1C developer|
| Dima|Senior Fullstack ...|
|Anton|Middle C++ developer|
| Vlad|Senior Blockchain...|
+-----+--------------------+



In [86]:
df.groupBy('phone mark').mean('salary').pandas_api().dropna()

,phone mark,avg(salary)
0,Samsung,715.0
1,Iphone,4275.0


In [88]:
df.groupBy('phone mark').mean('salary').sort('avg(salary)', ascending=False).show()

+----------+-----------+
|phone mark|avg(salary)|
+----------+-----------+
|    Iphone|     4275.0|
|      NULL|     2460.0|
|   Samsung|      715.0|
+----------+-----------+

